In [ ]:
import torch
import mlflow
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier


datamodule = ImageClassificationData.from_folders(
    train_folder="../data-2/hymenoptera_data/train/",
    val_folder="../data-2/hymenoptera_data/val/",
    batch_size=2,
    transform_kwargs={"image_size": (196, 196), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

# 2. Build the task
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())


EXPERIMENT_NAME = "dl_model_chapter02"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter02"):
    trainer.finetune(model, datamodule=datamodule, strategy="freeze")
    trainer.test()

